In [1]:
import os 
import cv2
import random
import pickle
from tqdm import tqdm
from tensorflow.keras import utils
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam




%matplotlib inline

In [2]:

DATA_DIR = 'F:\study\sem 10\cvpr\VGG 16\dataset'
TRAIN_DATA_DIR = os.path.join(DATA_DIR, 'asl_alphabet_train/asl_alphabet_train')
TEST_DATA_DIR = os.path.join(DATA_DIR, 'asl_alphabet_train/asl_alphabet_test')



In [3]:
img_size = 32
CATEGORIES = []

for i in os.listdir(TRAIN_DATA_DIR):
    CATEGORIES.append(i)
    
print(CATEGORIES)

['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [4]:
images = []
labels = []


for c in CATEGORIES:
    path = os.path.join(TRAIN_DATA_DIR, c) # 'E:/DATASETS/classification/Face Mask Dataset/train/WithMask'
    class_num = CATEGORIES.index(c) # 0
    for img in tqdm(os.listdir(path)):
        try:
            img_array = cv2.imread(os.path.join(path, img))   # read the image
            img_resized = cv2.resize(img_array,(img_size, img_size) )  # resize the image
            images.append(img_resized) # [ [img, 0], [], [], [], ...., []]
            labels.append(class_num)
        except WException as e:
            pass


100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:02<00:00, 1041.25it/s]


In [5]:

images = np.array(images)
images = images.astype('float32')/255.0
labels = utils.to_categorical(labels)
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.1)
print(Y_train.shape)

(78300, 29)


In [25]:
print('Loaded', len(X_train),'images for training,','Train data shape =', X_train.shape)
print('Loaded', len(Y_train),'images for testing','Test data shape =', Y_train.shape)



Loaded 78300 images for training, Train data shape = (78300, 32, 32, 3)
Loaded 78300 images for testing Test data shape = (78300, 29)


In [11]:
gpu_device = tf.config.experimental.list_physical_devices('GPU')
print(f"Number of GPU = {len(gpu_device)}")
tf.config.experimental.set_memory_growth(gpu_device[0], True)

Number of GPU = 1


In [26]:
print( len(X_train),'images for training,','Train data shape =', X_train.shape)
print( len(Y_train),'images for training,','Train data shape =', Y_train.shape)
print(len(X_test),'images for testing','Test data shape =', X_test.shape)


78300 images for training, Train data shape = (78300, 32, 32, 3)
78300 images for training, Train data shape = (78300, 29)
8700 images for testing Test data shape = (8700, 32, 32, 3)


In [19]:
model = keras.Sequential([
    keras.Input(shape=(32,32,3)),
    layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(29, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 8, 8, 256)        

In [22]:
learning_rate = 0.0001
adam = Adam(lr=learning_rate)
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics='accuracy'
)

In [23]:
h=model.fit(X_train, Y_train, batch_size=128, epochs=5, validation_split=0.1)

Epoch 1/5
551/551 [==============================] - 27s 45ms/step - loss: 3.3680 - accuracy: 0.0350 - val_loss: 3.3675 - val_accuracy: 0.0341
Epoch 2/5
551/551 [==============================] - 24s 43ms/step - loss: 3.3674 - accuracy: 0.0339 - val_loss: 3.3675 - val_accuracy: 0.0318
Epoch 3/5
551/551 [==============================] - 23s 42ms/step - loss: 3.3674 - accuracy: 0.0338 - val_loss: 3.3676 - val_accuracy: 0.0318
Epoch 4/5
551/551 [==============================] - 24s 43ms/step - loss: 3.3674 - accuracy: 0.0358 - val_loss: 3.3676 - val_accuracy: 0.0337
Epoch 5/5
551/551 [==============================] - 23s 42ms/step - loss: 3.3674 - accuracy: 0.0344 - val_loss: 3.3676 - val_accuracy: 0.0318
